In [1]:
# 读取环境变量
import os
frontAddress = os.environ.get('CTP_FRONT_ADDRESS')
mdFrontAddress = os.environ.get('CTP_MD_FRONT_ADDRESS')
brokerID = os.environ.get('CTP_BROKER_ID')
userID = os.environ.get('CTP_USER_ID')
password = os.environ.get('CTP_PASSWORD')
assert frontAddress and mdFrontAddress and brokerID and userID and password

In [2]:
# 导入相关库函数
from pyctp import Trader,Md,struct,callback
from time import sleep
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas import DataFrame
from pprint import pprint

In [3]:
# 相关方法定义

def getDefaultInstrumentID(months=1):
    """
    获取一个可用的交易品种ID
    """
    return datetime.strftime(datetime.now() + relativedelta(months=months),"IF%y%m")

orderRefSeq = 0
def getOrderRef():
    '''
    获取OrderRef序列值
    '''
    global orderRefSeq
    orderRefSeq += 1
    return ('%12d' % orderRefSeq).replace(' ','0') # '000000000001'

def getInsertOrderField(direction,action,volume=1):
    """
    获取一个有效的建单数据格式
    """
    inputOrderField = struct.CThostFtdcInputOrderField()
    inputOrderField.BrokerID = brokerID
    inputOrderField.InvestorID = userID
    inputOrderField.InstrumentID = getDefaultInstrumentID()
    inputOrderField.OrderRef =  getOrderRef() #
    inputOrderField.UserID = userID
    inputOrderField.OrderPriceType = '1'     # 任意价

    # inputOrderField.Direction = '0'          # 买
    # inputOrderField.CombOffsetFlag = '0'     # 开仓
    if action == 'open':
        inputOrderField.CombOffsetFlag = '0'
        inputOrderField.Direction = {'buy': '0', 'sell': '1'}[direction]
    elif action == 'close':
        inputOrderField.CombOffsetFlag = '1'
        inputOrderField.Direction = {'buy': '1', 'sell': '0'}[direction]
    else:
        raise Exception(u'未知的操作方向')

    inputOrderField.CombHedgeFlag = '1'      # 投机
    inputOrderField.LimitPrice = 0           # 限价 0表不限制
    inputOrderField.VolumeTotalOriginal = volume  # 手数
    inputOrderField.TimeCondition = '1'      # 立即完成否则撤消
    inputOrderField.GTDDate = ''
    inputOrderField.VolumeCondition = '1'    # 成交类型  '1' 任何数量  '2' 最小数量 '3'全部数量
    inputOrderField.MinVolume = volume       # 最小数量
    inputOrderField.ContingentCondition = '1' # 触发类型 '1' 立即否则撤消
    inputOrderField.StopPrice = 0             # 止损价
    inputOrderField.ForceCloseReason = '0'    # 强平标识 '0'非强平
    inputOrderField.IsAutoSuspend = 0         # 自动挂起标识
    inputOrderField.BusinessUnit = ''         # 业务单元
    inputOrderField.RequestID = 1
    inputOrderField.UserForceClose = 0        # 用户强平标识
    inputOrderField.IsSwapOrder = 0           # 互换单标识
    return inputOrderField


# 相关回调函数的定义
def OnRspOrderInsert(**kwargs):
    errorId = kwargs['RspInfo']['ErrorID']
    errorMsg = kwargs['RspInfo']['ErrorMsg']
    print 'OnRspOrderInsert() : ErrorId=%d, ErrorMsg=%s' % (errorId,errorMsg)   

def OnErrRtnOrderInsert(**kwargs):
    errorId = kwargs['RspInfo']['ErrorID']
    errorMsg = kwargs['RspInfo']['ErrorMsg']
    print 'OnErrRtnOrderInsert() : ErrorId=%d, ErrorMsg=%s' % (errorId,errorMsg)   


def OnRtnOrder(**kwargs):
    orderStatus = kwargs['Data']['OrderStatus']
    statusMsg = kwargs['Data']['StatusMsg']
    print 'OnRtnOrder() : OrderStatus=%s, StatusMsg=%s' % (orderStatus,statusMsg)   


def OnRtnTrade(**kwargs):
    print 'OnRtnTrade() : 已调用，报单执行成功!'   
    pprint(kwargs['Data'])



In [4]:
# 初始化trader接口，并绑定回调
trader = Trader(frontAddress, brokerID, userID, password)
trader.bind(callback.OnRspOrderInsert,OnRspOrderInsert)
trader.bind(callback.OnErrRtnOrderInsert,OnErrRtnOrderInsert)
trader.bind(callback.OnRtnOrder,OnRtnOrder)
trader.bind(callback.OnRtnTrade,OnRtnTrade)


UUID('8dbdc9e2-4b99-11e5-a2b8-00163e0006ed')

In [8]:
# 开仓
data = getInsertOrderField('buy','open')
trader.ReqOrderInsert(data)
sleep(2)
print 

OnRtnOrder() : OrderStatus=a, StatusMsg=报单已提交
OnRtnOrder() : OrderStatus=a, StatusMsg=报单已提交
OnRtnOrder() : OrderStatus=0, StatusMsg=全部成交报单已提交
OnRtnTrade() : 已调用，报单执行成功!
{u'BrokerID': u'66666',
 u'BrokerOrderSeq': 57089,
 u'BusinessUnit': u'205103',
 u'ClearingPartID': u'2051',
 u'ClientID': u'01735367',
 u'Direction': u'0',
 u'ExchangeID': u'CFFEX',
 u'ExchangeInstID': u'IF1509',
 u'HedgeFlag': u'1',
 u'InstrumentID': u'IF1509',
 u'InvestorID': u'60003010',
 u'OffsetFlag': u'0',
 u'OrderLocalID': u'       39944',
 u'OrderRef': u'000000000004',
 u'OrderSysID': u'     1065959',
 u'ParticipantID': u'2051',
 u'Price': 2829,
 u'PriceSource': u'2',
 u'SequenceNo': 67914,
 u'SettlementID': 1,
 u'TradeDate': u'20150826',
 u'TradeID': u'      331874',
 u'TradeSource': u'0',
 u'TradeTime': u'10:33:10',
 u'TradeType': u'0',
 u'TraderID': u'205103',
 u'TradingDay': u'20150826',
 u'TradingRole': u'1',
 u'UserID': u'60003010',
 u'Volume': 1}



In [9]:
# 平仓
data = getInsertOrderField('buy','close')
trader.ReqOrderInsert(data)
sleep(2)
print 

OnRtnOrder() : OrderStatus=a, StatusMsg=报单已提交
OnRtnOrder() : OrderStatus=a, StatusMsg=报单已提交
OnRtnOrder() : OrderStatus=0, StatusMsg=全部成交报单已提交
OnRtnTrade() : 已调用，报单执行成功!
{u'BrokerID': u'66666',
 u'BrokerOrderSeq': 57474,
 u'BusinessUnit': u'205105',
 u'ClearingPartID': u'2051',
 u'ClientID': u'01735367',
 u'Direction': u'1',
 u'ExchangeID': u'CFFEX',
 u'ExchangeInstID': u'IF1509',
 u'HedgeFlag': u'1',
 u'InstrumentID': u'IF1509',
 u'InvestorID': u'60003010',
 u'OffsetFlag': u'1',
 u'OrderLocalID': u'       40233',
 u'OrderRef': u'000000000005',
 u'OrderSysID': u'     1072609',
 u'ParticipantID': u'2051',
 u'Price': 2826,
 u'PriceSource': u'1',
 u'SequenceNo': 68065,
 u'SettlementID': 1,
 u'TradeDate': u'20150826',
 u'TradeID': u'      333819',
 u'TradeSource': u'0',
 u'TradeTime': u'10:33:43',
 u'TradeType': u'0',
 u'TraderID': u'205105',
 u'TradingDay': u'20150826',
 u'TradingRole': u'1',
 u'UserID': u'60003010',
 u'Volume': 1}

